<a href="https://colab.research.google.com/github/albertotrunk/depth2video/blob/main/stable_diffusion_depth2video_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites


In [ ]:
#todo
#ffmpeg -start_number 0 -i input.%03d.png -c:v libx264 -r 24 -pix_fmt yuv420p output.mp4
#ffmpeg -i output.mp4 -filter:v "minterpolate=mi_mode=2" output.mp4 
#ffmpeg -i <input> -filter:v fps=30 <output>
#sampledata
#color.mp4
#depth

#GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


#memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [1]:
#%pip install ..

%pip install -U git+https://github.com/huggingface/transformers.git
%pip install -U git+https://github.com/huggingface/diffusers.git accelerate ftfy scipy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-lh0_xl63
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-lh0_xl63
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 182 kB 28.0 MB/s 
     |████████████████████████████████| 7.6 MB 97.6 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5961994 sha256=859c018af991ba8d6531e290ede15579afef16de005896ad3cf2fa91c4ff0468
  Stored in directory: /tmp/pip-ephem-wheel-cache-f0_7c_ek/wheels/05/0a/97/64ae47c27ba95fae2cb5838e7b4b7247a34d4a8ba5f7092de2
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hugging

# Basic img2depth
Main example img2depth

In [14]:
import torch
import requests
from PIL import Image
from diffusers import StableDiffusionDepth2ImgPipeline

pipe = StableDiffusionDepth2ImgPipeline.from_pretrained(
   "stabilityai/stable-diffusion-2-depth",
   torch_dtype=torch.float16,
).to("cuda")

init_image = "https://user-images.githubusercontent.com/8300565/207172663-d347b44f-bce3-4a15-9e92-5411ccdd7cb4.png" #@param {type:"string"}
init_image = Image.open(requests.get(init_image, stream=True).raw)

init_image

prompt = "scarlett johansson" #@param {type:"string"}
n_propmt = "bad, deformed, ugly, bad anotomy" #@param {type:"string"}
strength = 0.2  #@param {type:"slider", min:0, max:1, step:0.1}

image = pipe(prompt=prompt, image=init_image, negative_prompt=n_propmt, strength=strength).images[0]

image


FileNotFoundError: ignored

In [ ]:
# the object returned is a python generator
answers = stability_api.generate(
    prompt="houston, we are a 'go' for launch!",
    seed=34567, # if provided, specifying a random seed makes results deterministic
    steps=20, # defaults to 30 if not specified
)

# iterating over the generator produces the api response
for resp in answers:
    for artifact in resp.artifacts:
        if artifact.finish_reason == generation.FILTER:
            warnings.warn(
                "Your request activated the API's safety filters and could not be processed."
                "Please modify the prompt and try again.")
        if artifact.type == generation.ARTIFACT_IMAGE:
            img = Image.open('/content/sample_data/01_c.png')
            display(img)

# Intermediate usage

3. (new!) An "init" image can be provided for text-driven image modification. To demonstrate, we can convert the image we just generated to a colored pencil sketch

In [ ]:
answers = stability_api.generate(
    prompt="childrens crayon drawing of a rocket launch",
    init_image=img,
    seed=54321, # if we're passing in an image generated by SD, you may get better results by providing a different seed value than was used to generate the image
    start_schedule=0.6, # this controls the "strength" of the prompt relative to the init image
)

# iterating over the generator produces the api response
for resp in answers:
    for artifact in resp.artifacts:
        if artifact.finish_reason == generation.FILTER:
            warnings.warn(
                "Your request activated the API's safety filters and could not be processed."
                "Please modify the prompt and try again.")
        if artifact.type == generation.ARTIFACT_IMAGE:
            img2 = img
            display(img2)

In [ ]:
# we can make a rough mask by thresholding the grayscaled image

import numpy as np

img2_grayscale = Image.open('/content/sample_data/01_m.png').convert('L')
img2_a = np.array(img2_grayscale)

mask = np.array(img2_grayscale)
strength = .2  # this controls the "strength" of the prompt relative to the init image

d = int(255 * (1-strength))
mask *= 255-d # convert from range [0,1] to [0,255]
mask += d

mask = Image.fromarray(mask)
mask

In [ ]:
# fuzzing the mask edges generally improves synthesis results

from torchvision.transforms import GaussianBlur

blur = GaussianBlur(11,20)
mask = blur(mask)
mask

In [ ]:
# now, let's send our drawing into space
answers = stability_api.generate(
    prompt="scarlett johansson",
    init_image=img2,
    mask_image=mask,
    seed=12345, # if we're passing in an image generated by SD, you may get better results by providing a different seed value than was used to generate the image
    start_schedule=1,
)

for resp in answers:
    for artifact in resp.artifacts:
        if artifact.finish_reason == generation.FILTER:
            warnings.warn(
                "Your request activated the API's safety filters and could not be processed."
                "Please modify the prompt and try again.")
        if artifact.type == generation.ARTIFACT_IMAGE:
            img3 = Image.open(io.BytesIO(artifact.binary))
            display(img3)

In [ ]:
# we can improve our image result by introducing clip guidance into the request. this takes longer (and costs more compute) but often yields better results.
# clip guidance is built with several presets optimized for speed or quality, but clip can be controlled more precisely with a variety of other params

answers = stability_api.generate(
    prompt="houston, we are a 'go' for launch!",
    seed=34567,  
    steps=35, # minimum of 35 steps recommended when using CLIP
    guidance_preset = generation.GUIDANCE_PRESET_FAST_BLUE
)

for resp in answers:
    for artifact in resp.artifacts:
        if artifact.finish_reason == generation.FILTER:
            warnings.warn(
                "Your request activated the API's safety filters and could not be processed."
                "Please modify the prompt and try again.")
        if artifact.type == generation.ARTIFACT_IMAGE:
            img4 = Image.open(io.BytesIO(artifact.binary))
            print('GUIDANCE: FAST-BLUE:')
            display(img4)

answers = stability_api.generate(
    prompt="houston, we are a 'go' for launch!",
    seed=34567, 
    steps=35, 
    guidance_preset = generation.GUIDANCE_PRESET_SLOWER
)

for resp in answers:
    for artifact in resp.artifacts:
        if artifact.finish_reason == generation.FILTER:
            warnings.warn(
                "Your request activated the API's safety filters and could not be processed."
                "Please modify the prompt and try again.")
        if artifact.type == generation.ARTIFACT_IMAGE:
            img5 = Image.open(io.BytesIO(artifact.binary))
            print('GUIDANCE: SLOWER:')
            display(img5)